# Factor Analysis using the CAPM and Fama-French Factor models

The main idea in Factor Analysis is to take a set of observed returns and decompose it into a set of explanatory returns.

First, we'll use the returns of Axis Long Term Equity Fund - Direct Plan - Growth Option

In [2]:
import pandas as pd

In [3]:
import Basic_Risk_Assessment_Tools as brat

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
dateparse = lambda x: pd.datetime.strptime(x, '%d-%m-%Y')

axis_nav = pd.read_excel("Data/Axis_NAV.xlsx", parse_dates=True, date_parser=dateparse,index_col=0)
axis_nav.sort_index(ascending=True, inplace=True)
axis_rets_d = axis_nav.pct_change().drop(list(axis_nav.index)[0])
axis_rets_d.columns = ['Returns']
axis_rets_d.tail()

,Returns
Date,
2020-07-17,0.009082
2020-07-20,0.015138
2020-07-21,0.002479
2020-07-22,-0.006807
2020-07-23,0.006902


To convert these to monthly returns we use the `.resample` method, which allows us to run an aggregation function on each group of returns in a time series.

We want to compound the returns, and we use the `compound` function in our toolkit.

In [6]:
axis_rets_m = axis_rets_d.resample('M').apply(brat.compound).to_period('M')
axis_rets_m.tail()

,Returns
Date,
2020-03,-0.210893
2020-04,0.107913
2020-05,-0.034474
2020-06,0.053615
2020-07,0.038697


To get the Factors for the analysis we will use the Nifty Indices.

NIFTY 500 as $Market Return$

NIFTY 50 as $Large Cap Returns$

NIFTY100 LOWVO30 as $Low Volatility Port Returns$

NIFTY50 VALUE 20 as $Value Stock Portfolio$


In [18]:
from nsepy import get_history
import datetime

In [41]:
start_date = datetime.date(2013,1,1)
end_date = datetime.date.today()
nse_tickers = ['NIFTY 500','NIFTY 50','NIFTY100 LOWVOL30','NIFTY50 VALUE 20']
closing_prices, returns = brat.get_returns_from_close_as_dataframe(nse_tickers,start_date,end_date, index=True)

NIFTY 500
done :)
NIFTY 50
done :)
NIFTY100 LOWVOL30
done :)
NIFTY50 VALUE 20
done :)


In [98]:
factors_d = pd.concat(returns, axis=1)
factors_d.columns = factors_d.columns.droplevel(-1)
factors_d.columns = ['Market','Large Cap','Low Vol','Value']
factors_d.head()

,Market,Large Cap,Low Vol,Value
2013-01-02,0.006832,0.007125,0.004749,0.006515
2013-01-03,0.003486,0.002711,0.000921,0.004983
2013-01-04,0.001716,0.001107,0.004106,0.003820
2013-01-07,-0.002591,-0.004613,-0.003719,-0.002089
2013-01-08,0.001345,0.002221,0.003933,0.001668


In [99]:
factors_d['Large Cap'] = factors_d['Large Cap']-factors_d['Market']
factors_d['Value'] = factors_d['Value']-factors_d['Market']
factors_d['Low Vol'] = factors_d['Low Vol']-factors_d['Market']
factors_d.head()

,Market,Large Cap,Low Vol,Value
2013-01-02,0.006832,0.000293,-0.002083,-0.000317
2013-01-03,0.003486,-0.000775,-0.002565,0.001497
2013-01-04,0.001716,-0.000610,0.002390,0.002104
2013-01-07,-0.002591,-0.002022,-0.001128,0.000502
2013-01-08,0.001345,0.000876,0.002588,0.000322


We subtract the $Market Returns$ from the $Large Cap$, $Low Vol$ and $Value$ columns to try and remove the effect of the Market Momentum and isolate the impact of Large Cap, Low Vol and Value on returns.

In [100]:
factors_m = factors_d.resample('M').apply(brat.compound).to_period('M')
factors_m.head()

,Market,Large Cap,Low Vol,Value
2013-01,0.001901,0.012067,0.006229,0.050005
2013-02,-0.066273,0.010228,0.014859,0.010166
2013-03,-0.008744,0.006872,0.006933,0.000417
2013-04,0.045828,-0.002014,0.021293,-0.024027
2013-05,0.008553,0.001029,-0.009929,0.004789


In [101]:
repo = pd.read_excel("Data/India_Repo_Rates.xlsx", parse_dates=True, index_col=0)
repo = repo/1200
repo = repo.resample('M').apply(brat.compound).to_period('M')['2013':]
repo.head()

,Rate
Date,
2013-01,0.007500
2013-02,0.007292
2013-03,0.007083
2013-04,0.007083
2013-05,0.006875


We will subtract the monthly repo return, which is taken as the risk free return, from the market return to get the $Excess Market Return$.

In [102]:
factors_m['Market'] = factors_m['Market']-repo['Rate']
factors_m.head()

,Market,Large Cap,Low Vol,Value
2013-01,-0.005599,0.012067,0.006229,0.050005
2013-02,-0.073565,0.010228,0.014859,0.010166
2013-03,-0.015827,0.006872,0.006933,0.000417
2013-04,0.038745,-0.002014,0.021293,-0.024027
2013-05,0.001678,0.001029,-0.009929,0.004789


To decompose the observed Axis Portfolio Returns into the portion that's due to the market and the rest that is not due to the market, using the CAPM as the explanatory model.

i.e.

$$ R_{axis,t} - R_{f,t} = \alpha + \beta(R_{mkt,t} - R_{f,t}) + \epsilon_t $$

We can use the `stats.api` for the linear regression as follows:

In [90]:
import statsmodels.api as sm
import numpy as np

In [112]:
axis_excess = axis_rets_m['Returns'] - repo['Rate']
mkt_excess = factors_m.loc["2013-01":"2020-07",['Market']]
exp_var = mkt_excess.copy()
exp_var["Constant"] = 1
lm = sm.OLS(axis_excess, exp_var).fit()

In [113]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     614.5
Date:                Sun, 26 Jul 2020   Prob (F-statistic):           1.00e-41
Time:                        12:14:54   Log-Likelihood:                 244.07
No. Observations:                  91   AIC:                            -484.1
Df Residuals:                      89   BIC:                            -479.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Market         0.8509      0.034     24.789      0.000       0.783       0.919
Constant       0.0061      0.002      3.469      0.001       0.003       0.010
==============================================================================
Omnibus:                        0.899   Durbin-Watson:                   1.627
Prob(Omnibus):                  0.638   Jarque-Bera (JB):                1.008
Skew:                          -0.200   Prob(JB):                        0.604
Kurtosis:                       2.674   Cond. No.                         19.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### The CAPM benchmark interpretation

This implies that the CAPM benchmark consists of 15 paisa in T-Bills and 85 paisa in the market. i.e. each rupee in the Axis portfolio is equivalent to 15 paisa in T-Bills and 85 paisa in the market. Relative to this, the Axis Asset Managers are adding (i.e. has $\alpha$ of) 0.61% _(per month!)_ although the degree of statistica significance is not very high.

Adding more factors....

In [114]:
exp_var["Size"] = factors_m.loc["2013-01":"2020-07",['Large Cap']]
exp_var["Value"] = factors_m.loc["2013-01":"2020-07",['Value']]
exp_var.head()

,Market,Constant,Size,Value
2013-01,-0.005599,1,0.012067,0.050005
2013-02,-0.073565,1,0.010228,0.010166
2013-03,-0.015827,1,0.006872,0.000417
2013-04,0.038745,1,-0.002014,-0.024027
2013-05,0.001678,1,0.001029,0.004789


In [115]:
lm = sm.OLS(axis_excess, exp_var).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.891
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     236.1
Date:                Sun, 26 Jul 2020   Prob (F-statistic):           1.11e-41
Time:                        12:14:56   Log-Likelihood:                 250.69
No. Observations:                  91   AIC:                            -493.4
Df Residuals:                      87   BIC:                            -483.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Market         0.7813      0.037     20.840      0.000       0.707       0.856
Constant       0.0065      0.002      3.915      0.000       0.003       0.010
Size          -0.2040      0.218     -0.935      0.352      -0.638       0.230
Value         -0.2311      0.086     -2.681      0.009      -0.402      -0.060
==============================================================================
Omnibus:                        1.960   Durbin-Watson:                   1.641
Prob(Omnibus):                  0.375   Jarque-Bera (JB):                1.592
Skew:                          -0.162   Prob(JB):                        0.451
Kurtosis:                       2.438   Cond. No.                         135.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### The Fama-French Benchmark Interpretation

The alpha has risen from .61% to about 0.65% per month. The loading on the market has moved lower from 0.85 to 0.78, which means that adding these new explanatory factors did change things.

We can interpret the loadings on Value being negative as saying that Axis has a significant Growth tilt - which should not be a shock because the portfolio that I actually used was $Axis Long Term Equity Fund - Direct Plan - Growth$ $Option$. 

Additionally, the negative tilt on size suggests that Axis tends to invest in small companies, not large companies.

The new way to interpret each dollar invested in Axis is: 78 cents in the market, 22 cents in Bills, 23 cents in Growth stocks and short 23 in Value Stocks, short 20 cents in LargeCap stocks and long 20 cents in Mid/SmallCap stocks. If we did all this, we would still end up underperforming Hathaway by about 65 basis points per month.

In [116]:
exp_var["Low Vol"] = factors_m.loc["2013-01":"2020-07",['Low Vol']]

In [117]:
lm = sm.OLS(axis_excess, exp_var).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     191.3
Date:                Sun, 26 Jul 2020   Prob (F-statistic):           6.20e-42
Time:                        12:18:35   Log-Likelihood:                 254.30
No. Observations:                  91   AIC:                            -498.6
Df Residuals:                      86   BIC:                            -486.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Market         0.8450      0.043     19.459      0.000       0.759       0.931
Constant       0.0060      0.002      3.749      0.000       0.003       0.009
Size          -0.2359      0.211     -1.117      0.267      -0.656       0.184
Value         -0.2915      0.086     -3.376      0.001      -0.463      -0.120
Low Vol        0.3263      0.123      2.663      0.009       0.083       0.570
==============================================================================
Omnibus:                        2.366   Durbin-Watson:                   1.650
Prob(Omnibus):                  0.306   Jarque-Bera (JB):                1.617
Skew:                           0.084   Prob(JB):                        0.446
Kurtosis:                       2.369   Cond. No.                         135.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Adding Volatility to the mix, we also see that the portfolio has a positive loading on Low Vol stocks with an aplha of 0.32%.